# Word Variable Choose in Python

This code is main edited by Enze Wang

Fixed by Yicen Liu and Hua Tong

# Attention

Please put four .csv file in data folder and put this code in main folder instead of code folder.

This code want to do two things:

1. We want to select noun words related to food and service which is important for our pizza business to raise suggestion and judge positive and negative for review including them.

2. We want to select important adj and adv words which show strong positive and negative meanning from review. Especially those with price, food taste and service.

3. Find important negative adverb. Like "not" or "no', which will inverse review attitude such as "not good". 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import itertools
from collections import Counter
from nltk import FreqDist
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')
from wordcloud import WordCloud
from nltk import sent_tokenize, word_tokenize, PorterStemmer, pos_tag 
from nltk.corpus import stopwords
from nltk.text import TextCollection
from nltk.corpus import wordnet
import nltk.tokenize as nt
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /Users/enze/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/enze/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/enze/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/enze/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
important_word_tf_idf=pd.read_csv("data/Importance_word_tf_idf.csv")
important_word_linear=pd.read_csv("data/Importance_matrix1.csv")
important_word_tree1 =pd.read_csv("data/Importance_matrix2.csv")
important_word_tree2 =pd.read_csv("data/Importance_matrix3.csv")

In [3]:
important_word_tf_idf

,Unnamed: 0,Word,Frequency,tf_idf
0,0,pizza,98227,0.015001
1,1,good,40352,0.011402
2,3,food,35067,0.010822
3,2,place,37016,0.010805
4,5,time,27434,0.010486
...,...,...,...,...
5510,5476,polished,31,0.000073
5511,5480,delightfully,31,0.000073
5512,5481,oversized,31,0.000073
5513,5482,landmark,31,0.000073


In [4]:
important_word_linear[0:60]

,Unnamed: 0,Feature,Weight,Importance
0,1,worst,-0.647437,-0.647437
1,2,advertise,-0.588695,-0.588695
2,3,horrible,-0.494203,-0.494203
3,4,unfriendly,-0.468956,-0.468956
4,5,screwed,-0.466549,-0.466549
5,6,existent,-0.461994,-0.461994
6,7,sucks,-0.460110,-0.460110
7,8,terrible,-0.459854,-0.459854
8,9,lacks,-0.456093,-0.456093
9,10,ill,-0.454989,-0.454989


# Attention:  Code running for long time below:

In [5]:
important_word_tree1[0:60]

,Unnamed: 0,Feature,Gain,Cover,Frequency,Importance
0,1,delicious,0.073103,0.016625,0.023030,0.073103
1,2,great,0.072575,0.023945,0.023750,0.072575
2,3,not,0.065984,0.018928,0.014754,0.065984
3,4,worst,0.045020,0.013206,0.006837,0.045020
4,5,best,0.043578,0.014173,0.010076,0.043578
5,6,rude,0.036553,0.011760,0.005398,0.036553
6,7,amazing,0.036305,0.013603,0.009356,0.036305
7,8,no,0.034660,0.006876,0.004678,0.034660
8,9,terrible,0.032316,0.013082,0.005038,0.032316
9,10,told,0.029601,0.004244,0.002519,0.029601


In [6]:
important_word_tree2[0:60]

,Unnamed: 0,Feature,Gain,Cover,Frequency,Importance
0,1,delicious,0.073103,0.016625,0.023030,0.073103
1,2,great,0.072575,0.023945,0.023750,0.072575
2,3,not,0.065984,0.018928,0.014754,0.065984
3,4,worst,0.045020,0.013206,0.006837,0.045020
4,5,best,0.043578,0.014173,0.010076,0.043578
5,6,rude,0.036553,0.011760,0.005398,0.036553
6,7,amazing,0.036305,0.013603,0.009356,0.036305
7,8,no,0.034660,0.006876,0.004678,0.034660
8,9,terrible,0.032316,0.013082,0.005038,0.032316
9,10,told,0.029601,0.004244,0.002519,0.029601


As you can see, the word importance ranked by tf-idf has little difference between the frequene rank. And rank from linear model and tree regression pay more attention on the adj, which show strong positive and negative attitude. This importance has great influence on the stars. 

# All Noun

We select all noun from important_word_tf_idf

In [7]:
def noun(i):
    if nltk.pos_tag(nt.word_tokenize(i))[0][1].startswith('N'):
        return True
    else:
        return False
    

In [8]:
important_word_tf_idf_most=important_word_tf_idf[important_word_tf_idf["Frequency"]>300]

In [9]:
Noun=[]
for i in important_word_tf_idf_most['Word']:
    if noun(i) == True:
        Noun.append(i)

In [10]:
len(Noun)

894

# Food Related Words

Here we select highly mentioned food related words from review, and they must occur more than 300 times. 

In [11]:
def food_related_word(i):
    syns = wn.synsets(str(i), pos = wn.NOUN)
    for syn in syns:
        if 'food' in syn.lexname():
            return True
    return False

In [12]:
Food_Word=[]
important_word_tf_idf_most=important_word_tf_idf[important_word_tf_idf["Frequency"]>100]
for i in important_word_tf_idf_most['Word']:
    if food_related_word(i) == True:
            Food_Word.append(i)

In [13]:
len(Food_Word)

331

In [14]:
Noun

['pizza',
 'food',
 'place',
 'time',
 'order',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'bar',
 'love',
 'table',
 'minute',
 'menu',
 'try',
 'night',
 'wait',
 'drink',
 'slice',
 'delivery',
 'staff',
 'price',
 'people',
 'bread',
 'eat',
 'thing',
 'hour',
 'area',
 'way',
 'dinner',
 'favorite',
 'experience',
 'meal',
 'pasta',
 'server',
 'friend',
 'right',
 'want',
 'lot',
 'think',
 'taste',
 'star',
 'flavor',
 'side',
 'day',
 'dish',
 'sandwich',
 'told',
 'lunch',
 'pie',
 'pepperoni',
 'customer',
 'sure',
 'bit',
 'location',
 'year',
 'recommend',
 'family',
 'review',
 'fry',
 'meatball',
 'everything',
 'excellent',
 'something',
 'quality',
 'option',
 'home',
 'half',
 'waitress',
 'wine',
 'awesome',
 'perfect',
 'worth',
 'tasty',
 'thin',
 'nothing',
 'burger',
 'visit',
 'selection',
 'pittsburgh',
 'top',
 'super',
 'sausage',
 'need',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'feel',
 'work',
 'call',
 'm

In [15]:
Food_Word

['pizza',
 'food',
 'cheese',
 'sauce',
 'salad',
 'chicken',
 'beer',
 'table',
 'delicious',
 'menu',
 'drink',
 'wing',
 'slice',
 'topping',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'taste',
 'side',
 'dish',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'bit',
 'special',
 'meatball',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'ingredient',
 'mushroom',
 'pepper',
 'sweet',
 'portion',
 'dough',
 'dessert',
 'soup',
 'steak',
 'dressing',
 'onion',
 'must',
 'white',
 'bite',
 'piece',
 'plate',
 'bacon',
 'game',
 'sub',
 'hoagie',
 'olive',
 'buffalo',
 'mozzarella',
 'water',
 'green',
 'potato',
 'cut',
 'joint',
 'entree',
 'veggie',
 'stick',
 'oil',
 'cream',
 'roll',
 'course',
 'shrimp',
 'fish',
 'gluten',
 'parmesan',
 'spinach',
 'beef',
 'egg',
 'date',
 'brunch',
 'ice',
 'breadstick',
 'breakfast',
 'cake',
 'split',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'gyro',
 'hoagy',
 'lasagna',
 'basil',
 'cocktail',
 'chip',
 'cri

In [16]:
Noun_Variables=Noun
for i in Food_Word:
    if i not in Noun:
        Noun_Variables.append(i)

In [17]:
Noun_Variables

['pizza',
 'food',
 'place',
 'time',
 'order',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'bar',
 'love',
 'table',
 'minute',
 'menu',
 'try',
 'night',
 'wait',
 'drink',
 'slice',
 'delivery',
 'staff',
 'price',
 'people',
 'bread',
 'eat',
 'thing',
 'hour',
 'area',
 'way',
 'dinner',
 'favorite',
 'experience',
 'meal',
 'pasta',
 'server',
 'friend',
 'right',
 'want',
 'lot',
 'think',
 'taste',
 'star',
 'flavor',
 'side',
 'day',
 'dish',
 'sandwich',
 'told',
 'lunch',
 'pie',
 'pepperoni',
 'customer',
 'sure',
 'bit',
 'location',
 'year',
 'recommend',
 'family',
 'review',
 'fry',
 'meatball',
 'everything',
 'excellent',
 'something',
 'quality',
 'option',
 'home',
 'half',
 'waitress',
 'wine',
 'awesome',
 'perfect',
 'worth',
 'tasty',
 'thin',
 'nothing',
 'burger',
 'visit',
 'selection',
 'pittsburgh',
 'top',
 'super',
 'sausage',
 'need',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'feel',
 'work',
 'call',
 'm

In [18]:
len(Noun_Variables)

1051

In [19]:
pd.DataFrame(Noun_Variables).to_csv('data/Noun_Variables.csv')

In [20]:
pizza_business_review=pd.read_csv('data/pizza_business_clean_review.csv')

After we write this Noun_Variables.csv, we select noun which we want to study. Some nouns such as street has almost no value to study. Those word list includes important food, service and other things related to business review. There are about 290 words.

In [21]:
Noun_Variables_Important=[
 'pizza',
 'food',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'menu',
 'delivery',
 'staff',
 'price',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'server',
 'flavor',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'location',
 'fry',
 'meatball',
 'waitress',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'manager',
 'style',
 'mushroom',
 'pepper',
 'dough',
 'dessert',
 'soup',
 'steak',
 'onion',
 'bartender',
 'bacon',
 'kitchen',
 'hoagie',
 'fan',
 'parking',
 'mozzarella',
 'employee',
 'water',
 'potato',
 'waiter',
 'entree',
 'oil',
 'cream',
 'serve',
 'shrimp',
 'fish',
 'gluten',
 'parmesan',
 'spinach',
 'beef',
 'egg',
 'brunch',
 'ice',
 'chef',
 'breadstick',
 'breakfast',
 'cake',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'bbq',
 'music',
 'lasagna',
 'gelato',
 'cocktail',
 'eggplant',
 'gnocchi',
 'butter',
 'ravioli',
 'taco',
 'pesto',
 'margherita',
 'chocolate',
 'coffee',
 'takeout',
 'management',
 'building',
 'tv',
 'ricotta',
 'soda',
 'rib',
 'calamari',
 'attitude',
 'crunchy',
 'arugula',
 'vegetable',
 'salmon',
 'salt',
 'artichoke',
 'margarita',
 'seafood',
 'byob',
 'crab',
 'prosciutto',
 'caesar',
 'veal',
 'bean',
 'grease',
 'creamy',
 'lemon',
 'honey',
 'spice',
 'noodle',
 'buffet',
 'worker',
 'bun',
 'tiramisu',
 'doughy',
 'panini',
 'lobster',
 'cider',
 'cooking',
 'diner',
 'environment',
 'zucchini',
 'tea',
 'donut',
 'broccoli',
 'feta',
 'ale',
 'balsamic',
 'ham',
 'turkey',
 'risotto',
 'napkin',
 'pretzel',
 'beverage',
 'recipe',
 'penne',
 'cheesecake',
 'lamb',
 'toast',
 'provolone',
 'fee',
 'pickle',
 'fat',
 'apple',
 'chili',
 'vodka',
 'cannoli',
 'caprese',
 'pita',
 'flatbread',
 'cherry',
 'cafe',
 'marsala',
 'sugar',
 'snack',
 'cauliflower',
 'scallop',
 'cookie',
 'wing',
 'topping',
 'dressing',
 'olive',
 'serving',
 'seasoning',
 'hummus',
 'pineapple',
 'corn',
 'cooky',
 'ketchup',
 'beet',
 'savory',
 'iceberg',
 'carbonara',
 'tuna',
 'truffle',
 'cucumber',
 'cuisine',
 'mustard',
 'cheddar',
 'cinnamon',
 'vinaigrette',
 'peanut',
 'coke',
 'rigatoni',
 'fig',
 'pancake',
 'orange',
 'juice',
 'antipasto',
 'strawberry',
 'patty',
 'cheeseburger',
 'wheat',
 'curd',
 'salami',
 'salsa',
 'breast',
 'diet',
 'slaw',
 'avocado',
 'fettuccine',
 'cracker',
 'carrot',
 'gorgonzola',
 'anchovy',
 'jam',
 'grape',
 'asparagus',
 'liquor',
 'pistachio',
 'tater',
 'sangria',
 'leg',
 'walnut',
 'candy',
 'jalapeno',
 'rosemary',
 'duck',
 'falafel',
 'brisket',
 'bisque',
 'polenta',
 'pastry',
 'oregano',
 'milk',
 'coleslaw',
 'loaf',
 'pumpkin',
 'almond',
 'hamburger',
 'flour',
 'chorizo',
 'martini',
 'ginger',
 'broth',
 'reuben',
 'linguine',
 'chipotle',
 'kale',
 'brownie',
 'crouton',
 'sushi',
 'blueberry',
 'ziti',
 'lemonade',
 'whiskey',
 'espresso',
 'blt',
 'pudding',
 'pear',
 'macaroni',
 'barbecue',
 'bleu',
 'dumpling',
 'vanilla',
 'bourbon',
 'tortilla',
 'syrup',
 'filet',
 'aioli',
 'supper',
 'raspberry',
 'tortellini',
 'cole',
 'biscuit',
 'condiment',
 'cod',
 'pepsi',
 'cranberry',
 'linguini',
 'fennel',
 'fryer',
 'pea',
 'coconut',
 'mixture',
 'peach',
 'manicotti',
 'romaine']

In [22]:
pd.DataFrame(Noun_Variables_Important).to_csv('data/Noun_Variables_Important.csv')

In [23]:
Review_List=[]
for i in pizza_business_review["clean_review"].tolist():
    Review_List.append(i.split(" "))

In [24]:
len(Review_List)

67156

In [25]:
Noun_Variables=Noun_Variables_Important
len(Noun_Variables)

287

We select more than 280 words. Build word matrix for important noun and all noun. 

# Attention: Code running for long time below:

In [26]:
Word_Matrix = np.zeros([len(Review_List), len(Noun_Variables)])
for i,x in enumerate(Review_List):
    for j,y in enumerate(Noun_Variables):
        Word_Matrix[i,j] = sum([z == y for z in x])

In [27]:
Word_Matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 2., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
Word_Matrix_Data=pd.DataFrame(Word_Matrix, columns=Noun_Variables)
pizza_business_review_word_matrix = pd.concat([pizza_business_review, Word_Matrix_Data], axis=1)

In [29]:
pizza_business_review_word_matrix

,Unnamed: 0,review_id,clean_review,pizza,food,cheese,sauce,service,salad,restaurant,...,cranberry,linguini,fennel,fryer,pea,coconut,mixture,peach,manicotti,romaine
0,0,hfXVxrlD6fr1kXfz6sR13Q,parking tough get get go much also get little ...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,EDQpLLFiAxeSxWI5ipwzBQ,server extensive knowledge wine various menu s...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,RWmT90rP7wYo57dG1cyvww,fast forward every trip nyc pitt includes visi...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0jduztItUhOh3FzKGjMxZQ,far favorite cleveland/akron restaurant great ...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,sxfmzhMvZ8LlI1x4fhNjKg,food good usually get italian sub pizza would ...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67151,67151,gcNRaJMDlSVs8YlqwpsRuw,originally side ohio always really enjoyed jet...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67152,67152,yFxDOdiDb8HPaK1io81kiw,yes yes yes loved going place shadyside ecstat...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67153,67153,Z_xmoLgIMB4As90t0a5eBQ,went lunch meeting amidst typical suburban str...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67154,67154,CzYRFCnkbqOVb3WIMMrbfw,wife eaten la brioche least dozen time food ne...,0.0,2.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


This is word matrix, where the entry is time word occurs in the review. For example, the first line review is:

In [30]:
pizza_business_review_word_matrix["clean_review"][0]

'parking tough get get go much also get little cautious sometimes frequently multiple questionable looking character pro legendary fry used super huge gave great value buck good burger drink look like lot different kind alcohol con customer service hahaha get fart one twin working ahead pack dirty messy table crap everywhere many questionable character someone asked wanted buy stolen jewelry eating totally honest french fry portion gotten smaller used need two basket large fry need one really expensive hot dog strict limit ketchup joking large fry anything le recommend getting legendary fry least recommend egg burger get go go light'

the first line of Word_Matrix with positive entry is:

In [31]:
Word_Matrix_Data.iloc[0,][Word_Matrix_Data.iloc[0,]>0]

service    1.0
fry        5.0
burger     2.0
parking    1.0
egg        1.0
ketchup    1.0
Name: 0, dtype: float64

In [32]:
Word_Matrix_Data=pd.concat([pizza_business_review["review_id"], Word_Matrix_Data], axis=1)
Word_Matrix_Data

,review_id,pizza,food,cheese,sauce,service,salad,restaurant,crust,chicken,...,cranberry,linguini,fennel,fryer,pea,coconut,mixture,peach,manicotti,romaine
0,hfXVxrlD6fr1kXfz6sR13Q,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,EDQpLLFiAxeSxWI5ipwzBQ,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,RWmT90rP7wYo57dG1cyvww,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0jduztItUhOh3FzKGjMxZQ,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,sxfmzhMvZ8LlI1x4fhNjKg,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67151,gcNRaJMDlSVs8YlqwpsRuw,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67152,yFxDOdiDb8HPaK1io81kiw,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67153,Z_xmoLgIMB4As90t0a5eBQ,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67154,CzYRFCnkbqOVb3WIMMrbfw,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
Word_Matrix_Data.to_csv("data/Word_Count_Matrix.csv")

# Positive & Negative Word

It is important to select Positive and Negative Words in our word list. Because we want to judge one review positive and negative in the following code, it is important to select the positive and negative words to judge it. 

The positive and negative word list is mainly from tree regression method word list, and add some highly frequent word from tf-idf. 

In [39]:
def ad(i):
    if nltk.pos_tag(nt.word_tokenize(i))[0][1].startswith('JJ'):
        return True
    else:
        return False

In [40]:
AD=[]
for i in important_word_tf_idf['Word']:
    if ad(i) == True:
        AD.append(i)
for i in important_word_tree2['Feature']:
    if ad(i) == True and i not in AD:
        AD.append(i)

In [41]:
len(AD)

448

In [42]:
pd.DataFrame(AD).to_csv("data/AD.csv")

Just like as noun, we translate and select by hand to find Positive_Word and Negative_Word which we can use in the review attitude judgement. In fact there is some very frequency word in tf-idf word list, which also have highly positive or negative attitude. We also include them. 

In [43]:
Positive_Word=[
 'acceptable',
 'actively',
 'adequate',
 'adorable',
 'adventurous',
 'affordable',
 'alive',
 'alright',
 'amazing',
 'appreciate',
 'appreciated',
 'appreciation',
 'attractive',
 'awesome',
 'beautiful',
 'beautifully',
 'beauty',
 'beloved',
 'best',
 'better',
 'bright',
 'brilliant',
 'care',
 'caring',
 'casual',
 'celebrate',
 'celebrated',
 'celebrating',
 'certainly',
 'charm',
 'charming',
 'cheap',
 'cheer',
 'cheerful',
 'chic',
 'classy',
 'clean',
 'clearly',
 'clever',
 'comfort',
 'comfortable',
 'comforting',
 'compliment',
 'complimentary',
 'complimented',
 'confident',
 'cool',
 'courteous',
 'courtesy',
 'creative',
 'curious',
 'cute',
 'definitely',
 'delectable',
 'delicious',
 'deliciously',
 'deliciousness',
 'delight',
 'delighted',
 'delightful',
 'delightfully',
 'divine',
 'easier',
 'easily',
 'easy',
 'efficient',
 'efficiently',
 'elegant',
 'energetic',
 'enjoy',
 'enjoyable',
 'enjoyed',
 'enjoying',
 'enjoys',
 'entertaining',
 'enthusiasm',
 'enthusiastic',
 'excellent',
 'excited',
 'excitement',
 'fabulous',
 'fan',
 'fantastic',
 'fav',
 'fave',
 'favor',
 'favorable',
 'favorite',
 'fine',
 'fit',
 'flawless',
 'fond',
 'fortunate',
 'free',
 'fresh',
 'friendlier',
 'friendliest',
 'friendly',
 'fun',
 'funny',
 'generous',
 'generously',
 'gift',
 'glad',
 'glorious',
 'good',
 'goodness',
 'gorgeous',
 'grace',
 'gracious',
 'graciously',
 'grand',
 'grateful',
 'great',
 'greater',
 'greatest',
 'greet',
 'greeted',
 'greeting',
 'ha',
 'haha',
 'hand',
 'happier',
 'happily',
 'happiness',
 'happy',
 'healthy',
 'heaven',
 'heavenly',
 'helpful',
 'helping',
 'highlight',
 'hilarious',
 'honest',
 'honestly',
 'hope',
 'hopeful',
 'hopefully',
 'hoping',
 'huge',
 'ideal',
 'importantly',
 'impressed',
 'impression',
 'impressive',
 'improved',
 'innovative',
 'interested',
 'interesting',
 'inviting',
 'joy',
 'kind',
 'kindly',
 'laugh',
 'laughing',
 'like',
 'liked',
 'liking',
 'lively',
 'lol',
 'love',
 'loved',
 'lovely',
 'lover',
 'loving',
 'loyal',
 'luck',
 'luckily',
 'lucky',
 'magnificent',
 'masterpiece',
 'miracle',
 'neat',
 'nice',
 'nicely',
 'nicer',
 'nicest',
 'ok',
 'okay',
 'opportunity',
 'original',
 'outstanding',
 'party',
 'passion',
 'passionate',
 'perfect',
 'perfection',
 'perfectly',
 'pleasant',
 'pleasantly',
 'please',
 'pleased',
 'pleaser',
 'pleasing',
 'pleasure',
 'popular',
 'positive',
 'praise',
 'pretty',
 'pride',
 'promising',
 'proud',
 'recommend',
 'recommended',
 'relaxed',
 'relaxing',
 'remarkable',
 'respect',
 'respectable',
 'respectful',
 'responsible',
 'romantic',
 'safe',
 'satisfaction',
 'satisfactory',
 'satisfied',
 'satisfy',
 'satisfying',
 'scrumptious',
 'sexy',
 'significant',
 'sincerely',
 'smart',
 'smile',
 'smiled',
 'smiling',
 'sparkling',
 'straight',
 'strong',
 'substantial',
 'succeed',
 'success',
 'successful',
 'super',
 'superb',
 'superior',
 'supreme',
 'surprising',
 'surprisingly',
 'sweet',
 'sweetness',
 'terrific',
 'thank',
 'thankful',
 'thankfully',
 'thanks',
 'thoughtful',
 'thrilled',
 'top',
 'truly',
 'unbelievable',
 'vibrant',
 'warm',
 'warmed',
 'warmer',
 'warmly',
 'warmth',
 'welcome',
 'welcomed',
 'welcoming',
 'winner',
 'winning',
 'wise',
 'wish',
 'wonderful',
 'wonderfully',
 'worth',
 'worthy',
 'wow',
 'wowed',
 'yay',
 'yummy']

In [44]:
Negative_Word=[
 'allergic',
 'angry',
 'annoyance',
 'annoyed',
 'annoying',
 'anxious',
 'appalled',
 'argue',
 'argued',
 'arguing',
 'arrogant',
 'ashamed',
 'attack',
 'avoid',
 'avoided',
 'avoiding',
 'awful',
 'awkwardly',
 'bad',
 'badly',
 'beaten',
 'biased',
 'bitch',
 'bitter',
 'blame',
 'blamed',
 'blind',
 'bloody',
 'bomb',
 'bored',
 'boring',
 'bother',
 'bothered',
 'bothering',
 'broke',
 'broken',
 'cancel',
 'cancelled',
 'chagrin',
 'chaos',
 'chaotic',
 'charged',
 'cheat',
 'clueless',
 'complain',
 'complained',
 'complaining',
 'complaint',
 'confused',
 'confusing',
 'confusion',
 'crappy',
 'crazy',
 'crime',
 'critical',
 'criticism',
 'crushed',
 'cry',
 'damn',
 'dangerous',
 'dead',
 'death',
 'deceiving',
 'delay',
 'delayed',
 'denied',
 'depressing',
 'desperate',
 'die',
 'died',
 'difficult',
 'difficulty',
 'dirt',
 'dirty',
 'disagree',
 'disappeared',
 'disappoint',
 'disappointed',
 'disappointing',
 'disappointment',
 'disappoints',
 'disaster',
 'disgusted',
 'disgusting',
 'dislike',
 'disorganized',
 'disrespect',
 'dissatisfied',
 'distracted',
 'doubt',
 'downside',
 'drop',
 'dull',
 'dumb',
 'embarrassed',
 'embarrassing',
 'error',
 'fail',
 'failed',
 'fails',
 'failure',
 'fake',
 'fallen',
 'fault',
 'fear',
 'fired',
 'fool',
 'fooled',
 'forced',
 'forget',
 'forgotten',
 'freaking',
 'frustrated',
 'frustrating',
 'frustration',
 'fuck',
 'guilty',
 'harsh',
 'hate',
 'hated',
 'hell',
 'hesitant',
 'hesitate',
 'hesitation',
 'hiding',
 'horrendous',
 'horrible',
 'horribly',
 'horrid',
 'horrific',
 'horror',
 'hurt',
 'idiot',
 'ignorant',
 'ignore',
 'ignored',
 'ignoring',
 'ill',
 'impatient',
 'incompetent',
 'inconvenience',
 'inconvenient',
 'insane',
 'insult',
 'insulting',
 'irritated',
 'jealous',
 'jerk',
 'killer',
 'lack',
 'lame',
 'lazy',
 'liar',
 'lied',
 'limited',
 'loose',
 'lose',
 'losing',
 'loss',
 'lost',
 'lousy',
 'low',
 'lower',
 'lying',
 'mad',
 'messed',
 'messy',
 'miserable',
 'misleading',
 'missed',
 'missing',
 'mistake',
 'nasty',
 'negative',
 'neglected',
 'nervous',
 'nonsense',
 'obnoxious',
 'offended',
 'offensive',
 'outrageous',
 'pain',
 'pathetic',
 'petty',
 'pissed',
 'poisoning',
 'poor',
 'questionable',
 'refuse',
 'refused',
 'regret',
 'regretted',
 'ridiculous',
 'ridiculously',
 'risk',
 'rob',
 'rotten',
 'rude',
 'rudely',
 'rudeness',
 'rudest',
 'ruin',
 'ruined',
 'sad',
 'sadly',
 'scam',
 'scared',
 'scary',
 'scream',
 'screamed',
 'screaming',
 'screwed',
 'severe',
 'severely',
 'shame',
 'shit',
 'shitty',
 'shock',
 'shocked',
 'shoot',
 'shortage',
 'shy',
 'sick',
 'skeptical',
 'smothered',
 'snob',
 'starving',
 'steal',
 'stingy',
 'stink',
 'stolen',
 'strange',
 'stressful',
 'struggle',
 'struggled',
 'stupid',
 'suck',
 'sucked',
 'sucker',
 'suffer',
 'terrible',
 'terribly',
 'threatening',
 'tired',
 'trouble',
 'ugh',
 'ugly',
 'unacceptable',
 'uncomfortable',
 'unfortunate',
 'unfortunately',
 'unfriendly',
 'unhappy',
 'unhealthy',
 'unimpressed',
 'unimpressive',
 'unpleasant',
 'unprofessional',
 'unsatisfied',
 'upset',
 'upsetting',
 'violation',
 'war',
 'warned',
 'warning',
 'waste',
 'wasted',
 'wasting',
 'weak',
 'worn',
 'worried',
 'worry',
 'worse',
 'worst',
 'wrong',
 'wtf']

# Important Positive & Negative Word

The important positive & negatiev words is different with positive & negative words above. The important one is that we can give clear suggestion. For example, "good" is not an important positive & negative words because we have no idea "good cheese" is good in price or good in taste. But delicious is an important positive & negative word, so as expensive, affordable.

The important Positive & Negative Word is mainly from positive & negative word list. 

In [45]:
Important_Positive_Word=[
"actively",
"affordable",
"casual",
"cheap",
"clean",
"comfortable",
"complimentary",
"courtesy",
"delectable",
"delicious",
"deliciously",
"deliciousness",
"efficient",
"efficiently",
"enthusiasm",
"enthusiastic",
"free",
"fresh",
"graciously",
"healthy",
"neat",
"respect",
"respectable",
"respectful",
"romantic",
"scrumptious",
"warmth",
]

In [46]:
Important_Negative_Word=[   
"awkwardly",
"bored",
"boring",
"chaotic",
"delay",
"delayed",
"depressing",
"dirt",
"disorganized",
"distracted",
"embarrassing",
"hesitation",
"ignore",
"ignoring",
"inconvenience",
"inconvenient",
"insult",
"insulting",
"lazy",
"outrageous",
"petty",
"rude",
"rudely",
"rudeness",
"rudest",
"shitty",
"shy",
"sick",
"snob",
"starving",
"stingy",
"uncomfortable",
"unhealthy",
"unprofessional",
"worn"  
]

In [47]:
pd.DataFrame(Important_Negative_Word,columns=["Word"]).to_csv("data/Important_Negative_Word.csv")
pd.DataFrame(Important_Positive_Word,columns=["Word"]).to_csv("data/Important_Positive_Word.csv")
pd.DataFrame(Negative_Word,columns=["Word"]).to_csv("data/Negative_Word.csv")
pd.DataFrame(Positive_Word,columns=["Word"]).to_csv("data/Positive_Word.csv")

# Negative Adverb

Why we need to find negative adverb, the reason is simple. In our following review attitude test, if there is a review like:

"It is not a good apple"

If we only consider the positive word "good", then we will regard it as a positive review, but in fact, it is negative. Therefore we will find any negative adverb in our review something like "not" and "no" before the positive or negative word.

In [48]:
pizza_review = pd.read_csv("data/pizza_review.csv")

In [49]:
pizza_review["text"]

0        Parking is tough to get here so I don't get to...
1        The server had an extensive knowledge of wines...
2        Fast Forward\n\nEvery trip from NYC to Pitt in...
3        By far my favorite Cleveland/Akron restaurant!...
4        All food is good here I usually get the Italia...
                               ...                        
67151    I'm originally from the other side of Ohio and...
67152    Yes, yes, YES!! We loved going to the place in...
67153    Went here for a lunch meeting amidst s typical...
67154    My wife and I have eaten at La Brioche at leas...
67155    First let me say Lindys in Columbus is one of ...
Name: text, Length: 67156, dtype: object

# Attention: Code running for long time below:

Run part of review to save time. 

In [50]:
RB=[]
for review in pizza_review["text"][0:10000].tolist():
    review=review.lower()
    review=nltk.word_tokenize(review)
    for j in review:
        if nltk.pos_tag(nt.word_tokenize(j))[0][1].startswith('RB'):
            RB.append(j)
    

In [51]:
from collections import Counter
Counter(RB)

Counter({'here': 3258,
         'so': 4871,
         "n't": 6818,
         'also': 2096,
         'sometimes': 228,
         'frequently': 40,
         'not': 6346,
         'more': 1844,
         'there': 3810,
         'ahead': 122,
         'everywhere': 47,
         'once': 470,
         'totally': 161,
         'too': 1771,
         'now': 808,
         'only': 1951,
         'really': 2466,
         'less': 298,
         'otherwise': 91,
         'forward': 153,
         'elsewhere': 91,
         'absolutely': 379,
         'far': 393,
         'very': 3715,
         'always': 1740,
         'else': 464,
         'usually': 468,
         'up': 2573,
         'back': 2504,
         'friendly': 1259,
         'still': 946,
         'just': 3438,
         'better': 1169,
         'enough': 719,
         'atmosphere': 693,
         'never': 1422,
         'again': 1382,
         'truly': 139,
         'definitely': 1255,
         'ever': 966,
         'pretty': 1256,
         'curren

In [52]:
RB_data=pd.DataFrame.from_dict(Counter(RB), orient='index')
RB_data=RB_data.sort_values(0,ascending=False)

In [53]:
RB_data[0:60]

,0
n't,6818
not,6346
so,4871
there,3810
very,3715
just,3438
here,3258
up,2573
back,2504
really,2466


In [54]:
RB_data[61:120]

,0
twice,199
recently,198
already,188
rather,184
completely,177
unfortunately,176
honestly,176
slightly,175
seriously,170
anywhere,167


As you can see, 

"n't" "not" "never" 

are three very important negative adverb which can put in the front of positive or negative words to inverse the attitude. In the review attitude function we will pay attention to this words.